<a href="https://colab.research.google.com/github/siddas18/Natural-Disaster-Damage-Prediction/blob/Kartik/NOAA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import requests
import gzip

In [ ]:
#Reading data file names
def get_file_names():
  URL = "https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/"

  r = requests.get(URL)
  df_list = pd.read_html(r.text) 
  df = df_list[0]

  df2 = df[df["Name"].str.contains("details",na=False)]
  df2 = df2['Name']
  return df2

#Call
df2 = get_file_names()


In [ ]:
noaa_list = []
parentURL = 'https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/'
for i in range(10):
  dwn = parentURL+df2.iloc[i,]
  with gzip.open(requests.get(dwn, stream=True).raw) as f:
      noaa_list.append(pd.read_csv(f))
      #print(noaa_list[0].columns)

#print(noaa_list)
noaa = pd.DataFrame(data=noaa_list[0],columns=noaa_list[0].columns)
len(noaa)

223

In [ ]:
parentURL = 'https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/'

def get_files(df2, parentURL):  
  noaa_list=[]
  for i in range(len(df2)):
    print(i)
    dwn = parentURL+df2.iloc[i,]
    with gzip.open(requests.get(dwn, stream=True).raw) as f:
      noaa_list.append(pd.read_csv(f))
  return noaa_list
noaa_list = get_files(df2,parentURL)      
df_merge = pd.concat(noaa_list)


In [ ]:
# Add the locations and fatalities data here before saving the df 

In [ ]:
# Save it to drive
df_merge.to_pickle('/content/drive/MyDrive/Projects/NOAA/Datasets/df_pickle.pkl')

In [ ]:
#Run
# Reading the persisted data from drive.No need to pull it again from the wibsite 
df_merge = pd.read_pickle('/content/drive/MyDrive/Projects/NOAA/Datasets/df_pickle.pkl')
del(noaa)

In [ ]:
df_merge.sort_values(by=['BEGIN_YEARMONTH'], inplace= True)

In [ ]:
# Need to pull all these from the website instead from drive.
#RUN 
#Earth surface temp merge
surface_temp = pd.read_csv("/content/drive/MyDrive/Projects/NOAA/Datasets/EPA Earth Surface temperature (1900-2020).csv",skiprows=7,usecols=[0,1], names = ["YEAR","Earth's surface(F)"])
# surface_temp.head()
df_merge = pd.merge(left=df_merge,right=surface_temp,how="left",on="YEAR")
# df_merge.head()

In [ ]:
#RUN 
#Green house ds merge
greenhouse = pd.read_csv("/content/drive/MyDrive/Projects/NOAA/Datasets/EPA Greenhouse Gas Emissions (1990-2019).csv",skiprows=6)
greenhouse.rename(columns={"Year":"YEAR"},inplace=True)
# greenhouse.head()
df_merge = pd.merge(left=df_merge,right=greenhouse,how="left",on="YEAR")
# df_merge.head()

In [ ]:
#RUN
#Precipitation ds merge
precipitation =  pd.read_csv('/content/drive/MyDrive/Projects/NOAA/Datasets/EPA Precipitation (1901-2020).csv',skiprows=6)
precipitation.rename(columns={"Year":"YEAR"},inplace=True)
df_merge = pd.merge(left=df_merge,right=precipitation,how="left",on="YEAR")
# df_merge.head()
df_merge.rename(columns={'Anomaly':'precipitation_Anomaly'},inplace=True)


In [ ]:
#RUN
#Sea level DS merge
sea_level = pd.read_csv('/content/drive/MyDrive/Projects/NOAA/Datasets/EPA Sea Level Readings (1880-2020).csv',skiprows=6,usecols=[0,1,4])
sea_level.rename(columns={"Year":"YEAR"},inplace=True)


#Using NOAA - Adjusted sea level (inches) for the year 1993-2019
sea_level.loc[sea_level['YEAR']>1992, 'CSIRO - Adjusted sea level (inches)'] = sea_level.loc[sea_level['YEAR']>1992, 'NOAA - Adjusted sea level (inches)']
sea_level.rename(columns={"CSIRO - Adjusted sea level (inches)":"sea_level_change"},inplace=True)
sea_level.drop('NOAA - Adjusted sea level (inches)',axis=1,inplace=True)

#Merge
df_merge = pd.merge(left=df_merge,right=sea_level,how="left",on="YEAR")


In [ ]:
#RUN
#Seasonal Temp anomaly DS merge
seasonal_temp = pd.read_csv('/content/drive/MyDrive/Projects/NOAA/Datasets/EPA seasonal-temperature(1896-2020).csv',skiprows=6)
seasonal_temp.rename(columns={"Year":"YEAR",'Winter':'winter_temp_anamoly','Spring':'spring_temp_anamoly','Summer':'summer_temp_anamoly','Fall':'fall_temp_anamoly'},inplace=True)
# seasonal_temp.head()

#Merge
df_merge = pd.merge(left=df_merge,right=seasonal_temp,how="left",on="YEAR")
# df_merge.head()

In [ ]:
#RUN
#artic_ice_cover DS merge
artic_ice_cover = pd.read_csv('/content/drive/MyDrive/Projects/NOAA/Datasets/arctic-sea-ice_fig-1.csv',skiprows=6)
artic_ice_cover.rename(columns={"Year":"YEAR",'March':'march_artic_ice_cover','September':'september_artic_ice_cover'},inplace=True)


#Merge
df_merge = pd.merge(left=df_merge,right=artic_ice_cover,how="left",on="YEAR")
# df_merge.head()

In [ ]:
#RUN
#Glaciers mass DS merge
glacier = pd.read_csv('/content/drive/MyDrive/Projects/NOAA/Datasets/glaciers_fig-1.csv',skiprows=6, usecols=[0,1])
glacier.rename(columns={"Year":"YEAR",'Mean cumulative mass balance': 'glacier_mass_balance'},inplace=True)
# glacier.head()

df_merge = pd.merge(left=df_merge,right=glacier,how="left",on="YEAR")
# df_merge.head()

In [ ]:
del(glacier,artic_ice_cover,seasonal_temp,sea_level,precipitation,greenhouse,surface_temp)

In [ ]:
#RUN
column_drop = ['END_AZIMUTH','CZ_FIPS','EPISODE_ID','EVENT_ID','STATE','STATE_FIPS','CZ_NAME','WFO','CZ_TIMEZONE','END_DATE_TIME','BEGIN_DATE_TIME','SOURCE','TOR_OTHER_WFO','TOR_OTHER_CZ_STATE','TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME','BEGIN_AZIMUTH','BEGIN_LOCATION','END_LOCATION','EPISODE_NARRATIVE', 'EVENT_NARRATIVE','DATA_SOURCE']
df_merge.drop(axis=1, columns=column_drop, inplace=True)
#df.drop(axis=1, columns='END_AZIMUTH', inplace=True)


### **Missing Values**

In [ ]:
# df = df_merge[(df_merge['YEAR']<2020) & (df_merge['YEAR']>=1990)]
# (df.isna().sum()/len(df))*100
#Checing of percentage of missing values per feature
(df_merge.isna().sum()/len(df_merge))*100

BEGIN_YEARMONTH               0.000000
BEGIN_DAY                     0.000000
BEGIN_TIME                    0.000000
END_YEARMONTH                 0.000000
END_DAY                       0.000000
END_TIME                      0.000000
YEAR                          0.000000
MONTH_NAME                    0.000000
EVENT_TYPE                    0.000000
CZ_TYPE                       0.000000
INJURIES_DIRECT               0.000000
INJURIES_INDIRECT             0.000000
DEATHS_DIRECT                 0.000000
DEATHS_INDIRECT               0.000000
DAMAGE_PROPERTY              31.506551
DAMAGE_CROPS                 37.955561
MAGNITUDE                    43.634970
MAGNITUDE_TYPE               76.126964
FLOOD_CAUSE                  94.897977
CATEGORY                     99.975288
TOR_F_SCALE                  95.978443
TOR_LENGTH                   84.756025
TOR_WIDTH                    84.756025
BEGIN_RANGE                  47.155502
END_RANGE                    47.177017
BEGIN_LAT                

In [ ]:
# Misiing values for the extra DS
# df_col = 'Carbon dioxide', 'Methane', 'Nitrous oxide', 'HFCs, PFCs, SF6, and NF3', 'precipitation_Anomaly', 'sea_level_change', 'winter_temp_anamoly', 'spring_temp_anamoly', 'summer_temp_anamoly', 'fall_temp_anamoly', 'march_artic_ice_cover','september_artic_ice_cover', 'glacier_mass_balance']
df_col = df_merge.columns.to_list()
df_col_sel = df_col[len(df_col)-14:]

#Check missing values per year for the extra datasets

group_year = df_merge.groupby(['YEAR'], dropna=False)
for i,col in enumerate(df_col_sel):
  plt.plot(group_year[col].apply(lambda x: x.isnull().sum()),label = col)
plt.plot(g_y, label = 'Total data points for the year')
# plt.legend()
# plt.ylim(0,20000)

In [ ]:
#Checking missin values in the target variable per year
# import seaborn as sns
# sns.heatmap(df.isnull(), cbar=False)

g_n = df_merge.groupby(['YEAR'], dropna=False)['DAMAGE_PROPERTY'].apply(lambda x: x.isnull().sum())
g_y = df_merge.groupby(['YEAR'], dropna=False).agg({'YEAR':'count'})
g_n2 = df_merge.groupby(['YEAR'], dropna=False)['DAMAGE_CROPS'].apply(lambda x: x.isnull().sum())
plt.plot(g_n, label = 'Missing target value for year')
plt.plot(g_y, label = 'Total data points for the year')
plt.plot(g_n2, label = 'crop')
plt.legend()



###Transforming colimns

In [ ]:
#Run

#Delete rows where 'DAMAGE_PROPERTY' has 'h','H'
# May be h/H means hundred. Lets try not to delete it. Changes to be made in strip_mul method
h_index = df_merge[df_merge['DAMAGE_PROPERTY'].str.contains('h', na=False)].index.to_list()
H_index = df_merge[df_merge['DAMAGE_PROPERTY'].str.contains('H', na=False)].index.to_list()

df_merge.drop(h_index, axis=0, inplace=True)
df_merge.drop(H_index, axis=0, inplace=True)


In [ ]:
df = df_merge.copy(deep=True)

In [ ]:
df.dtypes

BEGIN_YEARMONTH                int64
BEGIN_DAY                      int64
BEGIN_TIME                     int64
END_YEARMONTH                  int64
END_DAY                        int64
END_TIME                       int64
YEAR                           int64
MONTH_NAME                    object
EVENT_TYPE                    object
CZ_TYPE                       object
INJURIES_DIRECT                int64
INJURIES_INDIRECT              int64
DEATHS_DIRECT                  int64
DEATHS_INDIRECT                int64
DAMAGE_PROPERTY              float64
DAMAGE_CROPS                  object
MAGNITUDE                    float64
MAGNITUDE_TYPE                object
FLOOD_CAUSE                   object
CATEGORY                     float64
TOR_F_SCALE                   object
TOR_LENGTH                   float64
TOR_WIDTH                    float64
BEGIN_RANGE                  float64
END_RANGE                    float64
BEGIN_LAT                    float64
BEGIN_LON                    float64
E

In [ ]:
#RUN

# s = df_merge['DAMAGE_CROPS']
# s.str.strip().str[-1].unique()

#Convert Target column to nummeric
def strip_mul(x):
  # print(type(x))
  # print(x)
  if x is None:
    print(x, "  not processing")
    return x  
  else:
    if isinstance(x, str):
      if 'K' in x:
        return float(x.rstrip("K") or 1) * 1000
      if 'k' in x:
        return float(x.rstrip("k") or 1) * 1000
      elif 'M' in x:
        return float(x.rstrip("M") or 1) * 1000000
      elif 'B' in x:
        return float(x.rstrip("B") or 1) * 1000000000
      elif 'T' in x:
        # print(x)
        # print(float(x.rstrip("T") or 1) * 10**12)
        return float(x.rstrip("T") or 1) * 10**12
  
    
  
df['DAMAGE_PROPERTY'] = df['DAMAGE_PROPERTY'].map(strip_mul)
# df['DAMAGE_CROPS'] = df['DAMAGE_CROPS'].apply(strip_mul)



In [ ]:
#Cheking for null,inf values in DAMAGE_PROPERTY

# df_merge.isin([np.inf, -np.inf]).any()
df_merge['DAMAGE_PROPERTY'].isna().sum()/len(df_merge['DAMAGE_PROPERTY'])
# df_merge[df_merge['DAMAGE_PROPERTY']<0 or df_merge[]]

0.4293956136461046

Filling NA values in the DAMAGE_CROPS column

In [ ]:
#RUN
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df['DAMAGE_CROPS'] = df['DAMAGE_CROPS'].fillna(0)
df['DAMAGE_CROPS'].isna().sum()

0

In [ ]:
### Multiple issues with the datatype still exits. Please check